In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, GlobalMaxPooling2D, Dropout
from keras import optimizers , layers, applications
from tensorflow.keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
import tensorflow as tf
from keras.models import Model
from tf_explain.core.activations import ExtractActivations
from tensorflow.keras.applications.xception import decode_predictions
%matplotlib inline
from keras.models import Sequential
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

In [ ]:
train = ImageDataGenerator(rescale=1/255, shear_range=0.2, zoom_range=0.2,horizontal_flip=True)
valid = ImageDataGenerator(rescale=1/255,shear_range=0.2, zoom_range=0.2,horizontal_flip=True)

In [ ]:
train_set = train.flow_from_directory('H:/Course Problems/CSE-438/AutismDataset/train',
                                      target_size=(200,200), 
                                    class_mode='binary' )


In [ ]:
train_set.class_indices

In [ ]:
train_set.classes

In [ ]:
validation = valid.flow_from_directory('H:/Course Problems/CSE-438/AutismDataset/valid',
                                      target_size=(200,200),
                                    class_mode='binary' )

In [ ]:
validation.class_indices

In [ ]:
validation.classes

# Proposed Model

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(128,(3,3),activation="relu",input_shape=(200,200,3)),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    tf.keras.layers.Conv2D(256,(3,3),activation="relu"),
                                    tf.keras.layers.MaxPool2D(2,2),
                    
                                    tf.keras.layers.Flatten(),
                                    
                                    tf.keras.layers.Dense(512,activation="relu"), 
                                    tf.keras.layers.Dense(100,activation="relu"),  
                                                         
                                   tf.keras.layers.Dense(1,activation="sigmoid")
                                   ])

# VGG16 Model

In [ ]:
pre_trained_model = VGG16(input_shape=(200,200,3), include_top=False, weights="imagenet")

In [ ]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output

# Flatten the output layer to 1 dimension
x = GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

# VGG19 Model

In [ ]:
pre_trained_model = VGG19(input_shape=(200,200,3), include_top=False, weights="imagenet")

In [ ]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output

x = GlobalMaxPooling2D()(last_output)

x = Dense(512, activation='relu')(x)

x = Dropout(0.5)(x)

x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

# Xception Model

In [ ]:
##Xception
pre_model=tf.keras.applications.xception.Xception(weights='imagenet',include_top=True)

In [ ]:
last_layer = pre_model.get_layer('block14_sepconv2')
last_output = last_layer.output


x = GlobalMaxPooling2D()(last_output)

x = Dense(512, activation='relu')(x)

x = Dropout(0.5)(x)

x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_model.input, x)

# Compilation and Fitting

In [ ]:
model.compile(loss='binary_crossentropy',
  optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00001),
  metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs =5
model_fit = model.fit(train_set,
                     epochs=epochs,
                     validation_data= validation ,
                     callbacks=[
            tf.keras.callbacks.EarlyStopping(        
            patience=3,
            restore_best_weights=True)])

# Chart of accuracy and loss

In [ ]:
acc = model_fit.history['accuracy']
val_acc = model_fit.history['val_accuracy']

loss = model_fit.history['loss']
val_loss = model_fit.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Model use on Test Data

In [ ]:
dir_path = 'H:/Course Problems/CSE-438/AutismDataset/test'

for i in os.listdir(dir_path):
   img = image.load_img(dir_path+'//'+ i,target_size =(200,200))
   plt.imshow(img)
   plt.show()

   X = image.img_to_array(img)
   X = np.expand_dims(X, axis =0)
   images =np.vstack([X])
   val = model.predict(images)
   
   if 0.5>=val>=0:
      print("The child is Autistic")

   elif 1>=val>0.5:
      print("The child is not Autistic")
